In [53]:
import pandas as pd
cdf_path = "raw/anes_timeseries_cdf_csv_20220916.csv"
cdf = pd.read_csv(cdf_path)

/tmp/ipykernel_3554012/3633623264.py:3: DtypeWarning: Columns (4,5,6,7,8,9,13,14,17,18,19,21,22,23,24,25,26,27,28,29,30,31,32,33,34,38,39,40,41,42,43,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,233,254,275,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,361,382,403,424,425,426,427,428,429,430,431,

In [ ]:
def weighted_mean(row, val_col, weight_col):
    nomiss = row[row[val_col].notna()]
    return (nomiss[val_col] * nomiss[weight_col]).sum() / nomiss[weight_col].sum()

In [64]:
items = {
    "year": "VCF0004",
    "weight": "VCF0009z",
    "age": "VCF0101",
}

recode_mapping = {
    "female": {"item": "VCF0104", "mapping": {1: 0, 2: 1, 3: pd.NA, 0: pd.NA}},
    "race": {"item": "VCF0105a", "mapping": {1: "White", 2: "Black", 5: "Hispanic", 3: "Other", 4: "Other", 6: "Other", 7: "Other"}},
    "strong_republican": {"item": "VCF0301", "mapping": {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 1, 0: pd.NA}},
    "republican": {"item": "VCF0301", "mapping": {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 1, 7: 1, 0: pd.NA}},
    "democrat": {"item": "VCF0301", "mapping": {1: 1, 2: 1, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 0: pd.NA}},
    "lean_republican": {"item": "VCF0301", "mapping": {1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 1, 7: 1, 0: pd.NA}},
    "conservative": {"item": "VCF0803", "mapping": {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 1, 7: 1, 0: pd.NA}},
    "vote_rep_pres": {"item": "VCF0704a", "mapping": {1: 0, 2: 1, 0: pd.NA}},
    "prev_rep_pres": {"item": "VCF9027", "mapping": {1: 0, 2: 1, 0:pd.NA, 3: pd.NA, 5: pd.NA, 9: pd.NA}},
    "blacks_lazy": {"item": "VCF9271", "mapping": {-8: pd.NA, -9: pd.NA}},
    "therm_blacks": {"item": "VCF0206", "mapping": {98: pd.NA, 99: pd.NA}},
    "therm_whites": {"item": "VCF0207", "mapping": {98: pd.NA, 99: pd.NA}},
    "therm_police": {"item": "VCF0214", "mapping": {98: pd.NA, 99: pd.NA}},
    "party_3_narrow": {"item": "VCF0301", "mapping": {1: "D", 2: "D", 3: "I", 4: "I", 5: "I", 6: "R", 7: "R", 0: pd.NA}},
    "party_3_broad": {"item": "VCF0301", "mapping": {1: "D", 2: "D", 3: "D", 4: "I", 5: "R", 6: "R", 7: "R", 0: pd.NA}},
    "anti_imm": {"item": "VCF0879", "mapping": {8: pd.NA, 9: pd.NA}},
    "college": {"item": "VCF0110", "mapping": {4: 1, 0: pd.NA, 1: 0, 2: 0, 3:0}},
    "no_guar_jobs": {"item": "VCF0809", "mapping": {0: pd.NA, 9: pd.NA}},
    "distrust": {"item": "VCF0808", "mapping": {0: pd.NA, 1: 1, 2: 0}}
}

calculated = {
    "therm_delta": lambda row: row["therm_whites"] - row["therm_blacks"],
    "age_group": lambda row: (
        pd.NA if pd.isna(row["age"]) else
        pd.NA if row["age"] == " " else
        "18-29" if 18 <= int(row["age"]) <= 29 else
        "30-44" if 30 <= int(row["age"]) <= 44 else
        "45-64" if 45 <= int(row["age"]) <= 64 else
        "65+" if int(row["age"]) >= 65 else
        pd.NA
    ),
    "resentment": lambda row: (
        pd.NA if row["VCF9039"] == " " or row["VCF9040"] == " " or row["VCF9041"] == " " or row["VCF9042"] == " " else
        pd.NA if pd.isna(row["VCF9039"]) or pd.isna(row["VCF9040"]) or pd.isna(row["VCF9041"]) or pd.isna(row["VCF9042"]) else
        pd.NA if int(row["VCF9039"]) in (8,9) or int(row["VCF9040"]) in (8,9) or int(row["VCF9041"]) in (8,9) or int(row["VCF9042"]) in (8,9) else
        int(row["VCF9039"]) + int(row["VCF9042"]) - int(row["VCF9040"]) - int(row["VCF9041"]) #+ 12 # if you want a 4-20 scale. 
    ),
    "race_edu_block": lambda row: (
        pd.NA if pd.isna(row["college"]) else
        "NonWhite" if row["VCF0105a"] in (2,3,4,5,6,7) else
        "WhiteCollege" if row["VCF0105a"] == 1 and row["college"] == 1 else
        "WhiteNonCollege" if row["VCF0105a"] == 1 and row["college"] == 0 else
        pd.NA
    ),
    "race_party_block": lambda row: (
        "WhiteRep" if row["VCF0105a"] == 1 and row["VCF0301"] in (6,7) else
        "WhiteDem" if row["VCF0105a"] == 1 and row["VCF0301"] in (1,2) else
        "WhiteInd" if row["VCF0105a"] == 1 and row["VCF0301"] not in (1,2,6,7) else
        "NonWhite" if row["VCF0105a"] in (2,3,4,5,6,7) else
        pd.NA
    ),
    "vote_prev_vote": lambda row: (
        pd.NA if pd.isna(row["prev_rep_pres"]) or pd.isna(row["vote_rep_pres"]) else
        "Rep-Rep" if row["prev_rep_pres"] == 1 and row["vote_rep_pres"] == 1 else
        "Rep-Dem" if row["prev_rep_pres"] == 1 and row["vote_rep_pres"] == 0 else
        "Dem-Rep" if row["prev_rep_pres"] == 0 and row["vote_rep_pres"] == 1 else
        "Dem-Dem" if row["prev_rep_pres"] == 0 and row["vote_rep_pres"] == 0 else
        pd.NA
    )
}

In [63]:
anls = cdf.copy()

for item_name, var in items.items():
    anls[item_name] = anls[var]

for new_var, spec in recode_mapping.items():
    anls[new_var] = anls[spec["item"]].map(spec["mapping"])

for new_var, func in calculated.items():
    anls[new_var] = anls.apply(func, axis=1)

all_keys = list(items.keys()) + list(recode_mapping.keys()) + list(calculated.keys())
anls = anls[all_keys]
anls

,year,weight,age,female,race,strong_republican,republican,democrat,lean_republican,conservative,...,anti_imm,college,no_guar_jobs,distrust,therm_delta,age_group,resentment,race_edu_block,race_party_block,vote_prev_vote
0,1948,1.000000,,0,White,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,<NA>,<NA>,WhiteN,WhiteI,Dem-Dem
1,1948,1.000000,,1,White,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,<NA>,<NA>,WhiteN,WhiteI,Rep-Rep
2,1948,1.000000,,1,White,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,<NA>,<NA>,WhiteN,WhiteI,Dem-Dem
3,1948,1.000000,,1,White,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,<NA>,<NA>,WhiteN,WhiteI,Rep-Rep
4,1948,1.000000,,0,White,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,<NA>,<NA>,WhiteN,WhiteI,Dem-Dem
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68219,2020,1.052041,26,1,White,1,1,0,1,NaN,...,NaN,0,NaN,NaN,NaN,18-29,2,WhiteN,WhiteR,<NA>
68220,2020,1.124100,52,1,White,0,1,0,1,1,...,NaN,1,<NA>,NaN,NaN,45-64,-4,WhiteC,WhiteR,<NA>
68221,2020,1.514417,45,0,Hispanic,0,0,1,0,0,...,NaN,0,NaN,NaN,NaN,45-64,0,Nonwhite,Nonwhite,<NA>
68222,2020,0.292352,65,1,White,1,1,0,1,1,...,NaN,0,NaN,NaN,NaN,65+,2,WhiteN,WhiteR,Rep-Rep
